In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../')
from utils import *

pd.set_option('display.max_columns', None)

In [2]:
DATA_PATH = '../../data/'
PASSES_DF_PATH = DATA_PATH + 'passes_resp_df.pkl'
WYSCOUT_PATH = DATA_PATH + 'wyscout/'
SKILLCORNER_PATH = DATA_PATH + 'skillcorner/'
MATCH_IDS_PATH = DATA_PATH + 'matchids.csv'
XT_PLOT_PATH = DATA_PATH + 'smoothed_xt.csv'

In [3]:
passes_resp_df = pd.read_pickle(PASSES_DF_PATH)

Keep only players of the opposing team with responsibility greater than 0.

In [4]:
passes_resp_df = passes_resp_df[~passes_resp_df['tracking.is_teammate']]
df_resp_nonzero = passes_resp_df[passes_resp_df['responsibility'] > 0]

In [5]:
df_resp_nonzero = passes_resp_df[passes_resp_df['responsibility'] > 0]

In [6]:
match_info = pd.read_csv(MATCH_IDS_PATH)
match_info = match_info.dropna(subset=['wyscout', 'skillcorner'])
match_info['skillcorner'] = match_info['skillcorner'].astype(int)

wyscout_id = match_info['wyscout'].values[0]
skillcorner_id = match_info['skillcorner'].values[0]

In [7]:
metadata = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_metadata.csv")
tracking_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_tracking.csv")
lineup_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_lineup.csv")

In [8]:
defender_positions = ['Right Wing Back', 'Right Center Back', 'Left Center Back', 'Left Wing Back']

lineup_df['player_position'].unique()

array(['Right Wing Back', 'Right Winger', 'Attacking Midfield',
       'Left Midfield', 'Center Forward', 'Right Midfield',
       'Defensive Midfield', 'Right Forward', 'Left Center Back',
       'Left Wing Back', 'Goalkeeper', 'Right Center Back', 'Left Winger',
       'Left Forward'], dtype=object)

In [9]:

lineup_df_pos = lineup_df[['player_id', 'player_position']].rename(columns={'player_id': 'player.id.skillcorner'})

df_resp_def = df_resp_nonzero.merge(lineup_df_pos, how='left')

# check if every player has a matching id in lineup_df
assert len(df_resp_def) == len(df_resp_nonzero)

# get only defenders
df_resp_def = df_resp_def[df_resp_def['player_position'].isin(defender_positions)]

print(len(df_resp_def))

df_resp_def.head(5)

1427


,timestamp,period,matchId,team.name,player.id.wyscout,player.name,opponentTeam.name,pass.recipient.id.wyscout,pass.recipient.name,location.x,location.y,pass.endLocation.x,pass.endLocation.y,dxt,tracking.object_id,tracking.x,tracking.y,tracking.z,tracking.team_name,tracking.player.id.skillcorner,tracking.player_first_name,tracking.player_last_name,player.id.skillcorner,pass.recipient.id.skillcorner,tracking.is_teammate,tracking.is_self,responsibility,player_position
14,00:00:09.000000,1,5414111,Netherlands,476361,L. Geertruida,France,370,V. van Dijk,22.235294,7.046154,26.470588,-14.660000,-0.000660,12190,17.33,3.14,0.0,France,12190,Randal,Kolo Muani,14407,10738,False,False,0.443744,Right Center Back
15,00:00:09.000000,1,5414111,Netherlands,476361,L. Geertruida,France,370,V. van Dijk,22.235294,7.046154,26.470588,-14.660000,-0.000660,14407,22.60,7.11,0.0,Netherland,14407,Lutsharel,Geertruida,14407,10738,False,True,0.962982,Right Center Back
16,00:00:09.000000,1,5414111,Netherlands,476361,L. Geertruida,France,370,V. van Dijk,22.235294,7.046154,26.470588,-14.660000,-0.000660,10738,26.08,-16.74,0.0,Netherland,10738,Virgil,van Dijk,14407,10738,False,False,0.788364,Right Center Back
17,00:00:13.200000,1,5414111,Netherlands,370,V. van Dijk,France,476361,L. Geertruida,26.470588,-14.660000,26.470588,0.704615,0.000147,14407,27.22,0.54,0.0,Netherland,14407,Lutsharel,Geertruida,10738,14407,False,False,0.925059,Left Center Back
18,00:00:13.200000,1,5414111,Netherlands,370,V. van Dijk,France,476361,L. Geertruida,26.470588,-14.660000,26.470588,0.704615,0.000147,10738,22.84,-18.80,0.0,Netherland,10738,Virgil,van Dijk,10738,14407,False,True,0.449357,Left Center Back


In [10]:
# group by for each player_id_sk
# df_resp_def[['matchTimestamp', 'player_id_sk']]
columns_subnet = ['timestamp', 'responsibility', 'dxt', 'tracking.player.id.skillcorner', 'tracking.player_first_name', 'tracking.player_last_name', 'player_position', 'team.name']
columns_subnet_group = ['tracking.player.id.skillcorner', 'tracking.player_first_name', 'tracking.player_last_name', 'player_position', 'team.name']
# filter only relevant columns
df_resp_def = df_resp_def[columns_subnet]

# count number of timestamps for each player
# df_resp_def.groupby(columns_subnet).size()

# df_resp_def
df_resp_def.groupby(columns_subnet_group).size().reset_index(name='count') # groupby(columns_subnet).count('player_id_sk')

,tracking.player.id.skillcorner,tracking.player_first_name,tracking.player_last_name,player_position,team.name,count
0,277,Adrien,Rabiot,Left Center Back,Netherlands,18
1,277,Adrien,Rabiot,Left Wing Back,Netherlands,6
2,277,Adrien,Rabiot,Right Center Back,Netherlands,11
3,277,Adrien,Rabiot,Right Wing Back,Netherlands,14
4,946,Antoine,Griezmann,Left Center Back,Netherlands,20
...,...,...,...,...,...,...
165,33512,Kenneth,Taylor,Left Wing Back,Netherlands,11
166,33512,Kenneth,Taylor,Right Center Back,France,1
167,33512,Kenneth,Taylor,Right Center Back,Netherlands,7
168,33512,Kenneth,Taylor,Right Wing Back,France,1


In [11]:
df_resp_def_subnet = \
df_resp_def.groupby(columns_subnet_group).agg(
    average_responsibility=('responsibility', 'mean'),
    count=('timestamp', 'size')
).reset_index()

df_resp_def_subnet.head(10)

,tracking.player.id.skillcorner,tracking.player_first_name,tracking.player_last_name,player_position,team.name,average_responsibility,count
0,277,Adrien,Rabiot,Left Center Back,Netherlands,0.532752,18
1,277,Adrien,Rabiot,Left Wing Back,Netherlands,0.549800,6
2,277,Adrien,Rabiot,Right Center Back,Netherlands,0.294762,11
3,277,Adrien,Rabiot,Right Wing Back,Netherlands,0.404980,14
4,946,Antoine,Griezmann,Left Center Back,Netherlands,0.272557,20
5,946,Antoine,Griezmann,Left Wing Back,Netherlands,0.274135,10
6,946,Antoine,Griezmann,Right Center Back,Netherlands,0.434091,5
7,946,Antoine,Griezmann,Right Wing Back,Netherlands,0.300277,9
8,2106,Daley,Blind,Left Center Back,France,0.164668,1
9,2106,Daley,Blind,Left Center Back,Netherlands,0.200347,2


In [12]:
df_resp_def

,timestamp,responsibility,dxt,tracking.player.id.skillcorner,tracking.player_first_name,tracking.player_last_name,player_position,team.name
14,00:00:09.000000,0.443744,-0.000660,12190,Randal,Kolo Muani,Right Center Back,Netherlands
15,00:00:09.000000,0.962982,-0.000660,14407,Lutsharel,Geertruida,Right Center Back,Netherlands
16,00:00:09.000000,0.788364,-0.000660,10738,Virgil,van Dijk,Right Center Back,Netherlands
17,00:00:13.200000,0.925059,0.000147,14407,Lutsharel,Geertruida,Left Center Back,Netherlands
18,00:00:13.200000,0.449357,0.000147,10738,Virgil,van Dijk,Left Center Back,Netherlands
...,...,...,...,...,...,...,...,...
2612,01:32:30.600000,0.143328,0.002111,2106,Daley,Blind,Left Center Back,Netherlands
2613,01:32:30.600000,0.037622,0.002111,16320,Ibrahima,Konaté,Left Center Back,Netherlands
2614,01:32:30.600000,0.501371,0.002111,22103,Eduardo,Camavinga,Left Center Back,Netherlands
2615,01:32:30.600000,0.739960,0.002111,12229,Youssouf,Fofana,Left Center Back,Netherlands


### This table contains all recorded pass events. For each pass we include the total responsibility aggregated for all responsible defenders and the change in xT it incurs.
Columns: PassID, SenderID, StartingLocation, ReceiverID, EndingLocation, Duration, ΔxT, TotalResponsibility

In [13]:
passes_resp_df = pd.read_pickle(PASSES_DF_PATH)

In [14]:
passes_resp_df

,timestamp,period,matchId,team.name,player.id.wyscout,player.name,opponentTeam.name,pass.recipient.id.wyscout,pass.recipient.name,location.x,location.y,pass.endLocation.x,pass.endLocation.y,dxt,tracking.object_id,tracking.x,tracking.y,tracking.z,tracking.team_name,tracking.player.id.skillcorner,tracking.player_first_name,tracking.player_last_name,player.id.skillcorner,pass.recipient.id.skillcorner,tracking.is_teammate,tracking.is_self,responsibility
0,00:00:01.900000,1,5414111,Netherlands,118,M. Depay,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,7604,-41.03,-0.05,0.0,France,7604,Mike,Maignan,7387,7017,False,False,0.0
1,00:00:01.900000,1,5414111,Netherlands,118,M. Depay,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,16320,-15.86,-7.61,0.0,France,16320,Ibrahima,Konaté,7387,7017,False,False,0.0
2,00:00:01.900000,1,5414111,Netherlands,118,M. Depay,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,11516,-16.08,2.42,0.0,France,11516,Dayot,Upamecano,7387,7017,False,False,0.0
3,00:00:01.900000,1,5414111,Netherlands,118,M. Depay,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,11630,-14.54,-18.87,0.0,France,11630,Jules,Koundé,7387,7017,False,False,0.0
4,00:00:01.900000,1,5414111,Netherlands,118,M. Depay,France,625,M. de Roon,-1.058824,0.000000,8.470588,21.616842,-0.004162,11606,-14.08,17.56,0.0,France,11606,Theo,Hernández,7387,7017,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19641,01:33:38.800000,2,5414111,Netherlands,118,M. Depay,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,20191,31.49,-1.03,0.0,France,20191,Khéphren,Thuram-Ulien,7387,10916,False,False,0.0
19642,01:33:38.800000,2,5414111,Netherlands,118,M. Depay,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,12229,31.89,8.13,0.0,France,12229,Youssouf,Fofana,7387,10916,False,False,0.0
19643,01:33:38.800000,2,5414111,Netherlands,118,M. Depay,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,12246,35.56,12.33,0.0,France,12246,Moussa,Diaby,7387,10916,False,False,0.0
19644,01:33:38.800000,2,5414111,Netherlands,118,M. Depay,France,221069,W. Weghorst,-31.764706,-21.616842,-40.583333,-5.636923,0.064346,6028,5.09,-7.70,0.0,France,6028,Kylian,Mbappé Lottin,7387,10916,False,False,0.0


In [15]:
passes_resp_df.columns

Index(['timestamp', 'period', 'matchId', 'team.name', 'player.id.wyscout',
       'player.name', 'opponentTeam.name', 'pass.recipient.id.wyscout',
       'pass.recipient.name', 'location.x', 'location.y', 'pass.endLocation.x',
       'pass.endLocation.y', 'dxt', 'tracking.object_id', 'tracking.x',
       'tracking.y', 'tracking.z', 'tracking.team_name',
       'tracking.player.id.skillcorner', 'tracking.player_first_name',
       'tracking.player_last_name', 'player.id.skillcorner',
       'pass.recipient.id.skillcorner', 'tracking.is_teammate',
       'tracking.is_self', 'responsibility'],
      dtype='object')

In [16]:
passes_columns = ['matchId', 'timestamp', 'player.id.wyscout', 'player.name', 'location.x', 'location.y', 'pass.recipient.id.wyscout',
                 'pass.recipient.name', 'pass.endLocation.x', 'pass.endLocation.y', 'dxt', 'responsibility']
passes_network = passes_resp_df[passes_columns]

In [17]:
passes_network = passes_network.groupby('timestamp').agg({
    'matchId': 'first',
    'player.id.wyscout': 'first',
    'player.name': 'first',
    'location.x': 'first',
    'location.y': 'first',
    'pass.recipient.id.wyscout': 'first',
    'pass.recipient.name': 'first',
    'pass.endLocation.x': 'first',
    'pass.endLocation.y': 'first',
    'dxt': 'first',
    'responsibility': 'sum'  # Sum the responsibility column
}).rename(columns={'responsibility': 'Total_Responsibility'}).reset_index()

In [18]:
passes_network

,timestamp,matchId,player.id.wyscout,player.name,location.x,location.y,pass.recipient.id.wyscout,pass.recipient.name,pass.endLocation.x,pass.endLocation.y,dxt,Total_Responsibility
0,00:00:01.900000,5414111,118,M. Depay,-1.058824,0.000000,625,M. de Roon,8.470588,21.616842,-0.004162,2.713987
1,00:00:05.800000,5414111,625,M. de Roon,8.470588,21.616842,476361,L. Geertruida,22.235294,7.046154,-0.002211,2.268907
2,00:00:09.000000,5414111,476361,L. Geertruida,22.235294,7.046154,370,V. van Dijk,26.470588,-14.660000,-0.000660,2.195090
3,00:00:13.200000,5414111,370,V. van Dijk,26.470588,-14.660000,476361,L. Geertruida,26.470588,0.704615,0.000147,1.374416
4,00:00:17.900000,5414111,476361,L. Geertruida,19.058824,0.000000,370,V. van Dijk,21.176471,-21.616842,-0.000616,2.496181
...,...,...,...,...,...,...,...,...,...,...,...,...
888,01:33:18.500000,5414111,56,D. Klaassen,4.235294,10.382222,365446,D. Malen,10.588235,25.987368,-0.003139,0.610048
889,01:33:23.600000,5414111,365446,D. Malen,10.588235,25.987368,56,D. Klaassen,-11.647059,27.444211,0.008928,0.000000
890,01:33:24.900000,5414111,56,D. Klaassen,-11.647059,27.444211,221069,W. Weghorst,-13.764706,12.826667,0.006431,0.360404
891,01:33:36.200000,5414111,134102,N. Aké,-23.294118,-13.437778,118,M. Depay,-31.764706,-21.616842,0.013511,0.000000


In [ ]:
passes_network.to_csv('passes_network.csv')

In [19]:
df_resp_def

,timestamp,responsibility,dxt,tracking.player.id.skillcorner,tracking.player_first_name,tracking.player_last_name,player_position,team.name
14,00:00:09.000000,0.443744,-0.000660,12190,Randal,Kolo Muani,Right Center Back,Netherlands
15,00:00:09.000000,0.962982,-0.000660,14407,Lutsharel,Geertruida,Right Center Back,Netherlands
16,00:00:09.000000,0.788364,-0.000660,10738,Virgil,van Dijk,Right Center Back,Netherlands
17,00:00:13.200000,0.925059,0.000147,14407,Lutsharel,Geertruida,Left Center Back,Netherlands
18,00:00:13.200000,0.449357,0.000147,10738,Virgil,van Dijk,Left Center Back,Netherlands
...,...,...,...,...,...,...,...,...
2612,01:32:30.600000,0.143328,0.002111,2106,Daley,Blind,Left Center Back,Netherlands
2613,01:32:30.600000,0.037622,0.002111,16320,Ibrahima,Konaté,Left Center Back,Netherlands
2614,01:32:30.600000,0.501371,0.002111,22103,Eduardo,Camavinga,Left Center Back,Netherlands
2615,01:32:30.600000,0.739960,0.002111,12229,Youssouf,Fofana,Left Center Back,Netherlands


In [ ]:
df_resp_def.to_csv('defender_responsibility_network.csv')

In [20]:
df_resp_def

,timestamp,responsibility,dxt,tracking.player.id.skillcorner,tracking.player_first_name,tracking.player_last_name,player_position,team.name
14,00:00:09.000000,0.443744,-0.000660,12190,Randal,Kolo Muani,Right Center Back,Netherlands
15,00:00:09.000000,0.962982,-0.000660,14407,Lutsharel,Geertruida,Right Center Back,Netherlands
16,00:00:09.000000,0.788364,-0.000660,10738,Virgil,van Dijk,Right Center Back,Netherlands
17,00:00:13.200000,0.925059,0.000147,14407,Lutsharel,Geertruida,Left Center Back,Netherlands
18,00:00:13.200000,0.449357,0.000147,10738,Virgil,van Dijk,Left Center Back,Netherlands
...,...,...,...,...,...,...,...,...
2612,01:32:30.600000,0.143328,0.002111,2106,Daley,Blind,Left Center Back,Netherlands
2613,01:32:30.600000,0.037622,0.002111,16320,Ibrahima,Konaté,Left Center Back,Netherlands
2614,01:32:30.600000,0.501371,0.002111,22103,Eduardo,Camavinga,Left Center Back,Netherlands
2615,01:32:30.600000,0.739960,0.002111,12229,Youssouf,Fofana,Left Center Back,Netherlands


In [ ]:
df_resp_def_subnet.to_csv('defender_passes_subnetwork.csv')

In [21]:
df_resp_def_subnet

,tracking.player.id.skillcorner,tracking.player_first_name,tracking.player_last_name,player_position,team.name,average_responsibility,count
0,277,Adrien,Rabiot,Left Center Back,Netherlands,0.532752,18
1,277,Adrien,Rabiot,Left Wing Back,Netherlands,0.549800,6
2,277,Adrien,Rabiot,Right Center Back,Netherlands,0.294762,11
3,277,Adrien,Rabiot,Right Wing Back,Netherlands,0.404980,14
4,946,Antoine,Griezmann,Left Center Back,Netherlands,0.272557,20
...,...,...,...,...,...,...,...
165,33512,Kenneth,Taylor,Left Wing Back,Netherlands,0.440692,11
166,33512,Kenneth,Taylor,Right Center Back,France,0.413489,1
167,33512,Kenneth,Taylor,Right Center Back,Netherlands,0.436734,7
168,33512,Kenneth,Taylor,Right Wing Back,France,0.170796,1
